In [5]:
import numpy as np
import matplotlib.pyplot as plt

# Image data
I = np.array([
    [6, 9, 6, 5, 2, 3, 9],
    [7, 8, 3, 7, 4, 6, 1], 
    [2, 8, 5, 6, 9, 5, 4],
    [6, 7, 8, 1, 4, 6, 9],
    [1, 8, 8, 4, 6, 2, 6],
    [5, 2, 3, 7, 3, 6, 8]
])

def surface_detection(image, delta_x, cost_function):
    """
    Find optimal surface using dynamic programming
    
    Parameters:
    - image: 2D array of pixel values
    - delta_x: maximum allowed vertical movement between adjacent columns
    - cost_function: function to compute pixel cost (e.g., lambda x: abs(x-5))
    """
    rows, cols = image.shape
    
    # Initialize cost matrix
    costs = np.full((rows, cols), np.inf)
    
    # Initialize first column
    for r in range(rows):
        costs[r, 0] = cost_function(image[r, 0])
    
    # Fill remaining columns using dynamic programming
    for c in range(1, cols):
        for r in range(rows):
            pixel_cost = cost_function(image[r, c])
            
            # Check all valid previous positions within delta_x constraint
            for prev_r in range(max(0, r - delta_x), min(rows, r + delta_x + 1)):
                costs[r, c] = min(costs[r, c], costs[prev_r, c-1] + pixel_cost)
    
    # Backtrack to find optimal path
    path = []
    # Find minimum cost in last column
    last_row = np.argmin(costs[:, -1])
    path.append(last_row)
    
    # Backtrack from right to left
    for c in range(cols-2, -1, -1):
        current_row = path[-1]
        best_prev = current_row
        best_cost = np.inf
        
        for prev_r in range(max(0, current_row - delta_x), min(rows, current_row + delta_x + 1)):
            if costs[prev_r, c] < best_cost:
                best_cost = costs[prev_r, c]
                best_prev = prev_r
        
        path.append(best_prev)
    
    path.reverse()
    total_cost = costs[last_row, -1]
    
    return path, total_cost

def visualize_surface(image, path, title="Surface Detection Result"):
    """Visualize the image with the detected surface"""
    plt.figure(figsize=(10, 6))
    plt.imshow(image, cmap='viridis', aspect='auto')
    
    # Plot the surface path
    cols = range(len(path))
    plt.plot(cols, path, 'r-', linewidth=3, label='Detected Surface')
    plt.plot(cols, path, 'ro', markersize=8)
    
    # Add text annotations with pixel values
    for i, (col, row) in enumerate(zip(cols, path)):
        plt.text(col, row-0.3, str(image[row, col]), 
                ha='center', va='bottom', color='white', fontweight='bold')
    
    plt.colorbar(label='Pixel Values')
    plt.title(title)
    plt.xlabel('Column')
    plt.ylabel('Row')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# Test different configurations
def test_configuration(delta_x, cost_func, cost_name):
    print(f"\n{'='*50}")
    print(f"Testing: Δx = {delta_x}, Cost = {cost_name}")
    print(f"{'='*50}")
    
    try:
        path, total_cost = surface_detection(I, delta_x, cost_func)
        values = [I[path[i], i] for i in range(len(path))]
        
        print(f"Optimal path: {path}")
        print(f"Pixel values: {values}")
        print(f"Total cost: {total_cost}")
        
        # Check constraint violations
        max_jump = max(abs(path[i] - path[i-1]) for i in range(1, len(path)))
        print(f"Maximum jump: {max_jump}")
        print(f"Constraint satisfied: {max_jump <= delta_x}")
        
        return path, total_cost
        
    except Exception as e:
        print(f"Error: {e}")
        return None, None

# Example configurations to try
print("Image:")
print(I)

# Original options
test_configuration(3, lambda x: x, "I")
test_configuration(0, lambda x: 10 - x, "10 - I")  
test_configuration(1, lambda x: abs(x-5), "|I-5|")
test_configuration(2, lambda x: abs(x-5), "|I-5|")
test_configuration(1, lambda x: abs(x-6), "|I-6|")

# Uncomment to visualize (requires matplotlib)
# path_example, _ = surface_detection(I, 1, lambda x: abs(x-6))
# visualize_surface(I, path_example, "Δx=1, Cost=|I-6|")

print(f"\n{'='*50}")
print("Try your own configurations!")
print("Example usage:")
print("path, cost = surface_detection(I, delta_x=2, cost_function=lambda x: abs(x-7))")
print("values = [I[path[i], i] for i in range(len(path))]")
print("print(f'Path: {path}, Values: {values}, Cost: {cost}')")

Image:
[[6 9 6 5 2 3 9]
 [7 8 3 7 4 6 1]
 [2 8 5 6 9 5 4]
 [6 7 8 1 4 6 9]
 [1 8 8 4 6 2 6]
 [5 2 3 7 3 6 8]]

Testing: Δx = 3, Cost = I
Optimal path: [4, 5, 5, 3, 0, 0, np.int64(1)]
Pixel values: [np.int64(1), np.int64(2), np.int64(3), np.int64(1), np.int64(2), np.int64(3), np.int64(1)]
Total cost: 13.0
Maximum jump: 3
Constraint satisfied: True

Testing: Δx = 0, Cost = 10 - I
Optimal path: [3, 3, 3, 3, 3, 3, np.int64(3)]
Pixel values: [np.int64(6), np.int64(7), np.int64(8), np.int64(1), np.int64(4), np.int64(6), np.int64(9)]
Total cost: 29.0
Maximum jump: 0
Constraint satisfied: True

Testing: Δx = 1, Cost = |I-5|
Optimal path: [3, 3, 2, 2, 1, 2, np.int64(2)]
Pixel values: [np.int64(6), np.int64(7), np.int64(5), np.int64(6), np.int64(4), np.int64(5), np.int64(4)]
Total cost: 6.0
Maximum jump: 1
Constraint satisfied: True

Testing: Δx = 2, Cost = |I-5|
Optimal path: [5, 3, 2, 0, 1, 2, np.int64(2)]
Pixel values: [np.int64(5), np.int64(7), np.int64(5), np.int64(5), np.int64(4), np.int64

In [7]:
import numpy as np
import matplotlib.pyplot as plt

# Image data
I = np.array([
    [6, 9, 6, 5, 2, 3, 9],
    [7, 8, 3, 7, 4, 6, 1], 
    [2, 8, 5, 6, 9, 5, 4],
    [6, 7, 8, 1, 4, 6, 9],
    [1, 8, 8, 4, 6, 2, 6],
    [5, 2, 3, 7, 3, 6, 8]
])

def surface_detection(image, delta_x, cost_function):
    """Find optimal surface using dynamic programming"""
    rows, cols = image.shape
    
    # Initialize cost matrix
    costs = np.full((rows, cols), np.inf)
    
    # Initialize first column
    for r in range(rows):
        costs[r, 0] = cost_function(image[r, 0])
    
    # Fill remaining columns using dynamic programming
    for c in range(1, cols):
        for r in range(rows):
            pixel_cost = cost_function(image[r, c])
            
            # Check all valid previous positions within delta_x constraint
            for prev_r in range(max(0, r - delta_x), min(rows, r + delta_x + 1)):
                costs[r, c] = min(costs[r, c], costs[prev_r, c-1] + pixel_cost)
    
    # Backtrack to find optimal path
    path = []
    # Find minimum cost in last column
    last_row = np.argmin(costs[:, -1])
    path.append(last_row)
    
    # Backtrack from right to left
    for c in range(cols-2, -1, -1):
        current_row = path[-1]
        best_prev = current_row
        best_cost = np.inf
        
        for prev_r in range(max(0, current_row - delta_x), min(rows, current_row + delta_x + 1)):
            if costs[prev_r, c] < best_cost:
                best_cost = costs[prev_r, c]
                best_prev = prev_r
        
        path.append(best_prev)
    
    path.reverse()
    total_cost = costs[last_row, -1]
    
    return path, total_cost

def reverse_engineer_path(image, target_path):
    """
    🔍 REVERSE ENGINEER: Find delta_x and cost function that make target_path optimal
    
    Parameters:
    - image: The image matrix
    - target_path: List of row indices for each column [r0, r1, r2, ...]
    
    Returns:
    - List of matching configurations: [(delta_x, cost_func, name, cost), ...]
    """
    
    print(f"🔍 REVERSE ENGINEERING PATH: {target_path}")
    print("="*70)
    
    # Validate path length
    if len(target_path) != image.shape[1]:
        print(f"❌ ERROR: Path length {len(target_path)} doesn't match image columns {image.shape[1]}")
        return []
    
    # Validate path values
    rows = image.shape[0]
    for i, row in enumerate(target_path):
        if row < 0 or row >= rows:
            print(f"❌ ERROR: Invalid row {row} at column {i} (must be 0-{rows-1})")
            return []
    
    # Step 1: Analyze the path
    target_values = [image[target_path[i], i] for i in range(len(target_path))]
    
    # Find minimum required delta_x
    min_delta_x = 0
    jumps = []
    if len(target_path) > 1:
        for i in range(1, len(target_path)):
            jump = abs(target_path[i] - target_path[i-1])
            jumps.append(jump)
            min_delta_x = max(min_delta_x, jump)
    
    print(f"📊 PATH ANALYSIS:")
    print(f"   Path: {target_path}")
    print(f"   Values: {target_values}")
    print(f"   Jumps between columns: {jumps}")
    print(f"   Minimum required Δx: {min_delta_x}")
    
    # Step 2: Define cost functions to test
    cost_functions = [
        # Absolute distance functions
        (lambda x: abs(x - 1), "|I - 1|"),
        (lambda x: abs(x - 2), "|I - 2|"),
        (lambda x: abs(x - 3), "|I - 3|"),
        (lambda x: abs(x - 4), "|I - 4|"),
        (lambda x: abs(x - 5), "|I - 5|"),
        (lambda x: abs(x - 6), "|I - 6|"),
        (lambda x: abs(x - 7), "|I - 7|"),
        (lambda x: abs(x - 8), "|I - 8|"),
        (lambda x: abs(x - 9), "|I - 9|"),
        # Direct value functions
        (lambda x: x, "I"),
        (lambda x: -x, "-I"),
        (lambda x: 10 - x, "10 - I"),
        (lambda x: 15 - x, "15 - I"),
        # Squared functions
        (lambda x: (x - 5)**2, "(I - 5)²"),
        (lambda x: (x - 6)**2, "(I - 6)²"),
        (lambda x: x**2, "I²"),
    ]
    
    print(f"\n🎯 TESTING CONFIGURATIONS:")
    print("-"*70)
    
    exact_matches = []
    close_matches = []
    
    # Test different delta_x values (from minimum required up to reasonable limit)
    for delta_x in range(min_delta_x, min(5, rows)):
        print(f"\n   Δx = {delta_x}:")
        
        for cost_func, cost_name in cost_functions:
            try:
                # Find optimal path with this configuration
                optimal_path, optimal_cost = surface_detection(image, delta_x, cost_func)
                
                # Calculate cost of target path with this cost function
                target_cost = sum(cost_func(image[target_path[i], i]) for i in range(len(target_path)))
                
                # Check if paths match exactly
                if optimal_path == target_path:
                    print(f"      ✅ {cost_name}: EXACT MATCH! (cost = {target_cost:.1f})")
                    exact_matches.append((delta_x, cost_func, cost_name, target_cost))
                
                # Check if costs are the same (multiple optimal paths)
                elif abs(target_cost - optimal_cost) < 1e-10:
                    print(f"      🟡 {cost_name}: Same cost, different path (cost = {target_cost:.1f})")
                    print(f"         Optimal path: {optimal_path}")
                    close_matches.append((delta_x, cost_func, cost_name, target_cost, optimal_path))
                
                # Show cost difference for suboptimal paths
                else:
                    cost_diff = target_cost - optimal_cost
                    if cost_diff < 5:  # Only show if reasonably close
                        print(f"      ❌ {cost_name}: Not optimal (target: {target_cost:.1f}, optimal: {optimal_cost:.1f}, diff: +{cost_diff:.1f})")
                
            except Exception as e:
                # Skip configurations that cause errors
                continue
    
    # Step 3: Summary
    print(f"\n📋 SUMMARY:")
    print("="*70)
    
    if exact_matches:
        print(f"✅ Found {len(exact_matches)} EXACT match(es):")
        for i, (delta_x, cost_func, name, cost) in enumerate(exact_matches, 1):
            print(f"   {i}. Δx = {delta_x}, Cost function = {name}")
            print(f"      Total cost: {cost:.1f}")
            print(f"      Usage: surface_detection(I, {delta_x}, lambda x: {name.replace('I', 'x')})")
    
    if close_matches:
        print(f"\n🟡 Found {len(close_matches)} configuration(s) with same cost but different optimal path:")
        for i, (delta_x, cost_func, name, cost, opt_path) in enumerate(close_matches, 1):
            print(f"   {i}. Δx = {delta_x}, Cost = {name} (cost: {cost:.1f})")
            print(f"      Alternative optimal path: {opt_path}")
    
    if not exact_matches and not close_matches:
        print("❌ No exact matches found!")
        print("   Your path may not be optimal for any simple cost function.")
        print("   Try testing with more complex cost functions or check if path is valid.")
    
    return exact_matches

def quick_test_path(target_path):
    """Quick test function for interactive use"""
    return reverse_engineer_path(I, target_path)

# Interactive demonstration
print("🎮 INTERACTIVE REVERSE ENGINEERING DEMO")
print("="*60)
print("Image:")
print(I)

print("\n" + "="*80)
print("EXAMPLE 1: Original red diagonal line")
red_line = [3, 3, 2, 2, 1, 1, 2]
matches1 = reverse_engineer_path(I, red_line)

print("\n" + "="*80)
print("EXAMPLE 2: Horizontal line (row 2)")
horizontal = [2, 2, 2, 2, 2, 2, 2]
matches2 = reverse_engineer_path(I, horizontal)

print("\n" + "="*80)
print("EXAMPLE 3: Top row path")
top_row = [0, 0, 0, 0, 0, 0, 0]
matches3 = reverse_engineer_path(I, top_row)

print("\n" + "="*80)
print("🛠️  TRY YOUR OWN PATHS:")
print("Usage: reverse_engineer_path(I, [your_path_here])")
print("Or:    quick_test_path([your_path_here])")
print("\nExamples to try:")
print("• reverse_engineer_path(I, [0, 1, 2, 3, 4, 5, 4])  # Diagonal down then up")
print("• reverse_engineer_path(I, [5, 5, 5, 5, 5, 5, 5])  # Bottom row")
print("• reverse_engineer_path(I, [1, 2, 1, 2, 1, 2, 1])  # Zigzag pattern")
print("• reverse_engineer_path(I, [3, 4, 3, 2, 1, 0, 1])  # Custom path")

# Verification function
def verify_solution(delta_x, cost_func_str, expected_path):
    """Verify that a solution actually works"""
    print(f"\n🔬 VERIFICATION:")
    print(f"Testing Δx={delta_x}, Cost={cost_func_str}")
    
    # Create cost function from string (simplified parser)
    cost_funcs = {
        "|I-5|": lambda x: abs(x-5),
        "|I-6|": lambda x: abs(x-6),
        "I": lambda x: x,
        "10-I": lambda x: 10-x,
    }
    
    if cost_func_str in cost_funcs:
        path, cost = surface_detection(I, delta_x, cost_funcs[cost_func_str])
        print(f"Result: {path}")
        print(f"Expected: {expected_path}")
        print(f"Match: {path == expected_path}")
        return path == expected_path
    else:
        print(f"Cost function {cost_func_str} not recognized for verification")
        return False

print(f"\n📚 VERIFICATION EXAMPLE:")
print("verify_solution(1, '|I-6|', [3, 3, 2, 2, 1, 1, 2])")

🎮 INTERACTIVE REVERSE ENGINEERING DEMO
Image:
[[6 9 6 5 2 3 9]
 [7 8 3 7 4 6 1]
 [2 8 5 6 9 5 4]
 [6 7 8 1 4 6 9]
 [1 8 8 4 6 2 6]
 [5 2 3 7 3 6 8]]

EXAMPLE 1: Original red diagonal line
🔍 REVERSE ENGINEERING PATH: [3, 3, 2, 2, 1, 1, 2]
📊 PATH ANALYSIS:
   Path: [3, 3, 2, 2, 1, 1, 2]
   Values: [np.int64(6), np.int64(7), np.int64(5), np.int64(6), np.int64(4), np.int64(6), np.int64(4)]
   Jumps between columns: [0, 1, 0, 1, 0, 1]
   Minimum required Δx: 1

🎯 TESTING CONFIGURATIONS:
----------------------------------------------------------------------

   Δx = 1:
      ❌ |I - 5|: Not optimal (target: 7.0, optimal: 6.0, diff: +1.0)
      ❌ |I - 6|: Not optimal (target: 6.0, optimal: 4.0, diff: +2.0)
      ❌ (I - 5)²: Not optimal (target: 9.0, optimal: 8.0, diff: +1.0)
      ❌ (I - 6)²: Not optimal (target: 10.0, optimal: 6.0, diff: +4.0)

   Δx = 2:
      ❌ |I - 5|: Not optimal (target: 7.0, optimal: 4.0, diff: +3.0)
      ❌ |I - 6|: Not optimal (target: 6.0, optimal: 2.0, diff: +4.0)
 